In [1]:
import warnings

import wod_predictor
from wod_predictor.data_loader import DataLoader
from wod_predictor.splitter import DataSplitter
from wod_predictor.preprocessor import DataPreprocessor 
from wod_predictor.modeling import RandomForestModel

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_path = wod_predictor.__path__[0].replace("wod_predictor", "Data")
loader = DataLoader(root_path = data_path, objects= ['open_results','descriptions','benchmark_stats',  'athlete_info'])
data = loader.load()

In [4]:
splitter = DataSplitter(sample = 20000, test_ratio=  0.2, test_filter='23.*')
train_data, test_data = splitter.split(data)

In [5]:
# Example using MinMaxScaler
preprocessing_config = {
    'open_results':
    {
        'scale_up': True,
        'scale_args': {
            "method": "general",
            "scaler_name": "MinMaxScaler"
        }
    },
    'benchmark_stats':{
        'remove_outliers': True,
        'missing_method':'zero',
        'drop_missing_threshold': 0.9,
        'scale_args': {
            "method": "general",
            "scaler_name": "MinMaxScaler"
        }
        # 'neighbors': 5,
    },
    'athlete_info':{}
}
preprocessor = DataPreprocessor(config=preprocessing_config)
preprocessor.fit(data=train_data)


benchmark_df

In [6]:
train_preprocessed = preprocessor.transform(data=train_data)

Converted height to height in imperial units
Converted weight to weight in imperial units


In [7]:
test_preprocessed = preprocessor.transform(data=test_data)

Converted height to height in imperial units
Converted weight to weight in imperial units


In [8]:
config = {}
rf_modeler = RandomForestModel(config = config, n_estimators = 10)
rf_modeler.fit(X = train_preprocessed['X'], y = train_preprocessed['y'])
rf_modeler.predict(X = test_preprocessed['X'])
rf_modeler.show_results(y_test = test_preprocessed['y'], meta_data = test_preprocessed['meta_data'])

Mean Absolute Error: 0.04
Mean Absolute Percentage Error: 0.21


,y_test_mean,y_pred_mean,error_mean,error_percentage
23.1,185.289527,185.729175,7.975664,4.304433
23.1_scaled,222.000000,217.666667,32.266667,14.534535
23.2A,132.566925,132.238183,11.494523,8.670732
23.2B,208.337669,207.891078,18.532199,8.895271
23.3,26.929453,27.267228,9.953750,36.962318
23.3_scaled,10.186729,11.747680,3.762962,36.939842
